In [2]:
from astropy.table import Table, vstack, Column
from astropy.coordinates import SkyCoord, Angle

import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

In [8]:
df = Table.read('../catalogs/Wash/wash.vot')
df

WDS,Disc,Comp,Obs1,Obs2,pa1,pa2,sep1,sep2,mag1,mag2,DM,Notes,n_RAJ2000,RAJ2000,DEJ2000,_RA.icrs,_DE.icrs,recno
,,,yr,yr,deg,deg,arcsec,arcsec,mag,mag,,,,,,,,
str10,str7,str5,int16,int16,int16,int16,float32,float32,float32,float32,str9,str4,str1,str11,str11,str11,str11,int32
00000+7530,A 1248,,1904,1982,246,235,0.8,0.60,10.270,11.50,+74 1056,,,00 00 06.64,+75 28 59.8,00 00 06.64,+75 28 59.8,1
00000+4004,ES 2543,AB,1931,2015,252,253,4.8,4.40,12.100,13.10,,,,00 00 03.66,+40 05 19.4,00 00 03.66,+40 05 19.4,2
00000+4004,ES 2543,AC,1931,2015,69,66,20.0,14.40,12.100,14.10,,,,00 00 03.66,+40 05 19.4,00 00 03.66,+40 05 19.4,3
00000+3852,BU 860,,1881,2016,107,107,6.7,6.60,6.620,11.40,+38 5108,,,00 00 01.20,+38 51 33.4,00 00 01.20,+38 51 33.4,4
00000+0044,SKF1115,,2001,2015,33,30,0.8,1.00,17.800,17.80,,,,23 59 59.85,+00 43 32.7,23 59 59.85,+00 43 32.7,5
00000-0530,OCC 622,,1982,1982,-1,-1,0.1,0.10,9.500,9.70,-06 6337,,,00 00 00.08,-05 29 39.7,00 00 00.08,-05 29 39.7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23599+1413,SKF1113,,2001,2016,171,171,1.6,1.60,16.300,16.60,,,,23 59 56.44,+14 13 02.5,23 59 56.44,+14 13 02.5,156638


In [10]:
def ra_converter(ra_str):
    h = int(ra_str[0:2])
    m = int(ra_str[3:5])
    s = float(ra_str[6:])
    result = 15*h + 15* (m/60) + 15*(s/3600)
    return result

def dec_converter(dec_str):
    if(dec_str[0] == '+'):
        sign = 1
    else:
        sign = -1
    h = int(dec_str[1:3])
    m = int(dec_str[4:6])
    s = float(dec_str[7:])
    return sign * (h + (m/60) + (s/3600))

In [12]:
test_ra = ra_converter(df['RAJ2000'][0])
test_dec = dec_converter(df['DEJ2000'][0])

In [53]:
sky_obj = SkyCoord(ra = test_ra * u.deg, dec = test_dec * u.deg, frame = 'icrs')

In [55]:
coord1 = sky_obj.directional_offset_by(position_angle = df['pa1'][0], separation = df['sep1'][0])
coord2 = sky_obj.directional_offset_by(position_angle = df['pa2'][0], separation= df['sep2'][0])
coord1, coord2

(<SkyCoord (ICRS): (ra, dec) in deg
     (111.12453289, 51.09723457)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (26.72310223, 43.13730438)>)

In [47]:
list = np.array(df['RAJ2000'])
add_this_ra = []

ra_deg = np.vectorize(ra_converter)
add_this_ra = ra_deg(list)
add_this_ra

array([2.76666667e-02, 1.52500000e-02, 1.52500000e-02, ...,
       3.59963458e+02, 3.59975000e+02, 3.59973333e+02])

In [49]:
list = np.array(df['DEJ2000'])
add_this_dec = []

dec_deg = np.vectorize(dec_converter)
add_this_dec = dec_deg(list)
add_this_dec

array([ 75.48327778,  40.08872222,  40.08872222, ...,  -9.61236111,
       -18.3       , -31.20033333])

In [59]:
final_sky = SkyCoord(ra = add_this_ra * u.deg, dec = add_this_dec * u.deg, frame = 'icrs') 
final_sky

dtype = object
unit = deg,deg
class = SkyCoord
n_bad = 0
length = 156643


In [103]:
final_coord1 = final_sky.directional_offset_by(position_angle= df['pa1'], separation= df['sep1'])
final_coord2 = final_sky.directional_offset_by(position_angle= df['pa2'], separation= df['sep2'])

In [105]:
final_coord1

<SkyCoord (ICRS): (ra, dec) in deg
    [(111.12453289,  51.09723457), (312.51709333, -32.66918418),
     (201.09766492,  73.06277178), ..., (176.02508661,  41.41743135),
     (117.88221038,  -4.86387747), ( 20.01680136, -69.67962755)]>

In [14]:
#correct way

pairs = Table(names=['idx','ra','dec','sep','mag'],
              dtype=[int,float,float,float,float])

for i in range(len(df)):
    # if wds['Comp'][i] == '':
    split_ra = df['RAJ2000'][i].split(' ')
    split_dec = df['DEJ2000'][i].split(' ')
    ra = split_ra[0] + 'h' + split_ra[1] + 'm' + split_ra[2] + 's'
    dec = split_dec[0] + 'd' + split_dec[1] + 'm' + split_dec[2] + 's'

    c1 = SkyCoord(ra, dec, frame='icrs')
    
    position_angle_2 = df['pa2'][i] * u.deg
    separation_2 = df['sep2'][i] * u.arcsec
    
    c2 = c1.directional_offset_by(position_angle_2, separation_2)
    
    pairs.add_row([i, c1.ra.deg, c1.dec.deg, separation_2.value, 
                   df['mag1'][i]])
    pairs.add_row([i, c2.ra.deg, c2.dec.deg, separation_2.value, 
                   df['mag2'][i]]) 

In [15]:
pairs.write('../catalogs/Wash/wds.vot', format = 'votable', overwrite = True)

In [26]:
wds = Table.read('../catalogs/Wash/wds.vot')
wds

idx,ra,dec,sep,mag
int64,float64,float64,float64,float64
0,0.027666666666666662,75.48327777777779,0.6000000238418579,10.270000457763672
0,0.027122012254739414,75.48318218107309,0.6000000238418579,11.5
1,0.01525,40.08872222222222,4.400000095367432,12.100000381469727
1,0.013722238930403132,40.088364868984854,4.400000095367432,13.100000381469727
2,0.01525,40.08872222222222,14.399999618530273,12.100000381469727
2,0.020026521544045252,40.09034907066208,14.399999618530273,14.100000381469727
...,...,...,...,...
156639,359.9892029297497,-4.222314702439741,77.0999984741211,17.100000381469727
156640,359.96345833333334,-9.612361111111111,3.700000047683716,11.899999618530273


In [ ]:
new_df = Table.read('../catalogs/Wash/wds.vot')

In [ ]:
col = np.unique(df['source_id'])